<a href="https://colab.research.google.com/github/skaurl/NH-bigdata/blob/main/classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 55.0MB 102kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 11.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588521 sha256=5c6b05c6422493af8b0882d8267331628123219730355270ad4cf0c474f4fcfe
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 14.4MB/s 
     |████████████████████████████████| 757kB 13.2MB/s 
     |████████████████████████████████| 890kB 30.4MB/s 
     |████████████████████████████████| 3.0MB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=52ba6ad57f6a29eae7bfbbf935501d614ee37aaf387bcc1bc4378269532536ee
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ae3kh4th
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ae3kh4th
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=ddbbaa30c17239b33b04bc1d2a0ee4ffb213c339251c4a2e31d2fb33990e7bee
  Stored in directory: /tmp/pip-ephem-wheel-cache-hupdi38g/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
dataset_train = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_train_train.txt", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_train_test.txt", field_indices=[0,1], num_discard_samples=1)

In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
## Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.01
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7377938628196716 train acc 0.484375
epoch 1 batch id 201 loss 0.07376079261302948 train acc 0.9011971393034826
epoch 1 batch id 401 loss 0.011022021993994713 train acc 0.9371493142144638
epoch 1 batch id 601 loss 0.1214507445693016 train acc 0.9522670549084858
epoch 1 batch id 801 loss 0.12387747317552567 train acc 0.9606156367041199

epoch 1 train acc 0.9655966720917469


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9927778543849973


epoch 2 batch id 1 loss 0.02290971204638481 train acc 0.984375
epoch 2 batch id 201 loss 0.05441557615995407 train acc 0.9892723880597015
epoch 2 batch id 401 loss 0.0012571588158607483 train acc 0.9906483790523691
epoch 2 batch id 601 loss 0.0435289591550827 train acc 0.9919405158069884
epoch 2 batch id 801 loss 0.00036771452869288623 train acc 0.9925483770287141

epoch 2 train acc 0.9929946088264794


...


epoch 5 test acc 0.9956362612612613


epoch 6 batch id 1 loss 5.8277426433051005e-05 train acc 1.0
epoch 6 batch id 201 loss 5.120886999065988e-05 train acc 0.9998445273631841
epoch 6 batch id 401 loss 3.0937862902646884e-05 train acc 0.9998441396508728
epoch 6 batch id 601 loss 7.218721293611452e-05 train acc 0.9998700083194676
epoch 6 batch id 801 loss 2.9112608899595216e-05 train acc 0.9998829588014981

epoch 6 train acc 0.9998746238716149



epoch 6 test acc 0.9949324324324325


epoch 7 batch id 1 loss 3.2150408515008166e-05 train acc 1.0
epoch 7 batch id 201 loss 3.421225483180024e-05 train acc 0.9999222636815921
epoch 7 batch id 401 loss 3.1179984944174066e-05 train acc 0.9999220698254364
epoch 7 batch id 601 loss 3.968456439906731e-05 train acc 0.9999220049916805
epoch 7 batch id 801 loss 2.541345020290464e-05 train acc 0.9999414794007491

epoch 7 train acc 0.9999373119358074



epoch 7 test acc 0.995213963963964


In [23]:
dataset_test = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_test.txt", field_indices=[0,1], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=5)

model.eval()

Predict = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(out,1)
    Predict.append(int(predict[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [24]:
import pandas as pd

news_dataset_test = pd.read_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_test.csv")

del news_dataset_test["n_id"]
del news_dataset_test["date"]
del news_dataset_test["title"]
del news_dataset_test["ord"]

news_dataset_test["info"] = Predict

print(news_dataset_test.groupby("info").size())

news_dataset_test = news_dataset_test[["id","info"]]

news_dataset_test.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/result/sample_submission.csv", index=False)

info
0    87833
1    54732
dtype: int64
